In [ ]:
!pip install rouge-score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24932 sha256=7cbd255cdbe995cd6695957eb37b7303ad10b29fef37037c677893966628117a
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [ ]:
from rouge_score import rouge_scorer
import json
import inflect
import pandas as pd
pd.set_option("display.max_colwidth", None)
from itertools import product
from statistics import mean
import json
import inflect
import pandas as pd
import numpy as np

In [ ]:
def pairwise_rouge(df, column_name_1, column_name_2):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL',"rougeLsum"], use_stemmer=True)

    scores = []
    for i in range(len(df)):
        score = scorer.score(df.iloc[i][column_name_1], df.iloc[i][column_name_2])

        scores.append({
            "document": i,
            "rouge_1": score["rouge1"].fmeasure,
            "rouge_2": score["rouge2"].fmeasure,
            "rouge_L": score["rougeL"].fmeasure,
            "rouge_Lsum": score["rougeLsum"].fmeasure
        })

    return scores

def compute_rouge_for_models(dataframe):
    reference_models = ['gpt4']
    candidate_models = dataframe['model'].unique()
    reference_settings = ['top2_golden']
    candidate_settings = ['top2_golden']

    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL', 'rougeLsum'], use_stemmer=True)

    rouge_scores = []

    for model in candidate_models:
        if model not in reference_models:
            for setting in candidate_settings:
                # Filter rows for the candidate model and setting
                filtered_df = dataframe[(dataframe['model'] == model) & (dataframe['setting'] == setting)]

                # Compute ROUGE scores against the reference models and settings
                for ref_model in reference_models:
                    for ref_setting in reference_settings:
                        reference_text = dataframe[(dataframe['model'] == ref_model) & (dataframe['setting'] == ref_setting)]['concat_column'].values[0]
                        candidate_text = filtered_df['concat_column'].values[0]
                        scores = scorer.score(reference_text, candidate_text)

                        rouge_scores.append({
                            "candidate_model": model,
                            "candidate_setting": setting,
                            "reference_model": ref_model,
                            "reference_setting": ref_setting,
                            "rouge_1": scores["rouge1"].fmeasure,
                            "rouge_2": scores["rouge2"].fmeasure,
                            "rouge_L": scores["rougeL"].fmeasure,
                            "rouge_Lsum": scores["rougeLsum"].fmeasure
                        })

    return rouge_scores

In [ ]:
csv_files = ["task3_summary_trainset.csv", "task3_summary_devset.csv", "task3_summary_testset.csv"]
dfs = [pd.read_csv(file) for file in csv_files]
concatenated_df = pd.concat(dfs)
concatenated_df = concatenated_df.reset_index(drop=True)

In [ ]:
compute_rouge_for_models(concatenated_df)